# Enhancing Accuracy in Healthcare Record Interpretation with AI21 Models

In the context of healthcare, the veracity of information is paramount. Accurate data is crucial for informed decision-making, enabling healthcare professionals to deliver quality care, reduce misdiagnoses, and prevent adverse events. This notebook shows how to use AI21's [Contextual Answers](https://aws.amazon.com/marketplace/pp/prodview-gwbjdp3tmh3bw) model to derive grounded answers from healthcare documents. In this notebook, we focus on answering questions from patient healthcare records.

## Install and import dependencies

In [ ]:
!pip install  ai21

In [2]:
import json
import ai21
from ai21 import AI21Client
client = AI21Client()
#client = AI21Client(api_key='')

In [3]:
def call_ca(context,question):
    response = client.answer.create(
      context=context,
      question=question)
    answer=response.answer
    if response.answer is None:
        answer="None"
    return(answer)

# Ask Healthcare Questions Using Contextual Answers

Next, we will use Contextual Answers to address inquiries related to this specific patient's health records. Contextual Answers provides a platform that enables us to extract precise information from unstructured data by as natural language questions. Importantly, Contextual Answers will not only answer the questions provided, it will refuse to asnwer questions whose answers are not in the document. By strictly adhering to the text of the document, contextual answers provides increased safety and reliability.

In [4]:
patient_record_context='''
# Chief Complaint
No complaints.

# History of Present Illness
Timothy142 is a 6 year-old non-hispanic black male. His parents report that he sometimes has been coughing at night.

# Social History
 Patient has never smoked.

Patient comes from a middle socioeconomic background. Patient currently has Humana.

# Allergies
No Known Allergies.

# Medications
No Active Medications.

# Assessment and Plan


## Plan
Patient was given the following immunizations: influenza, seasonal, injectable, preservative free. 
The following procedures were conducted:
- medication reconciliation (procedure)
'''

### (Optional) Read from data in S3
In complex applications, healthcare data will frequently be stored in data lakes such as S3 object store. In this case, you can read the data from there prior to passing it to AI21's Contextual Answers model.




In [5]:
import boto3

def read_s3_object(s3_url):
    if s3_url.startswith('s3://'):
        s3_url = s3_url[5:]
        bucket_name, object_key = s3_url.split('/', 1)
    else:
        raise ValueError("URL must start with s3://")
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    return data

#s3_url = '' #in the format s3://BUCKET/directory/file.txt
#data = read_s3_object(s3_url)
#context=data.decode('utf-8')

## Answer Questions whose answers are in the document
First we will provide questions that can be directly answered from the document.

If the first invokation of the endpoint fails, **just simply rerun the cell**; the first invocation of the endpoint sometimes has an error.

In [6]:
questions=["How old is the patient?",
           "What insurance does the patient have?", 
           "What medical procedures were performed?"]

In [7]:

for question in questions:
    response = call_ca(
      context=patient_record_context,
      question=question,
)
    print(response)
        

The patient is 6 years old.
The patient has Humana insurance.
The medical procedures performed on Timothy142 were medication reconciliation and immunizations.


In [8]:
questions_not_in_doc=[
    "What are insurances similar to Humana?",
    "Does the patient's family have a history of heart disease?",
    "What are possible medications the patient should be taking?"]

In [9]:
for question in questions_not_in_doc:
    response = call_ca(
      context=patient_record_context,
      question=question,
     )
    print(response)

None
None
None


Note that Contextual Answers refuses to answer questions that are not in the document; even when there is a possiblity to deduce the answer based on facts outside of the document. For example, given that Timothy is only 6 years old, it is reasonable to assume that he is not competent to choose his own medication. Nevertheless, **because this fact is not explicity said in the document** Contextual Answers correctly does not return any answers. This strict grounding in the document text is desirable in many regulated industries, such as healthcare.